# Auswertungshilfe

Im Folgenden finden Sie Rezepte und kurze Anleitungen, die Ihnen bei der Auswertung der Daten helfen sollen. Sie können jedes beliebige Tool für die Auswertung verwenden, solange Sie zu einem brauchbaren Ergebnis kommen. Stark empfohlen wird die Verwendung von Python-Skripten.


## 1. Einlesen und Korrektur

Korrektur der Aufnahmen mittels der gewonnenen Bias-, Dark- und Flat-Aufnahmen (siehe bereitgestelltes PDF in Moodle). Hierfür können Python-Funktionen verwendet werden. Beispiele von einfachen Operationen, siehe auch die [Astropy Dokumentation](https://docs.astropy.org/en/stable/io/fits/).

Bias, Dark und Flat Korrektur werden an die Science Frames wie folgt angewandt:

$$
\mathrm{science frame} = \frac{\mathrm{raw science frame}- \mathrm{master dark} - \mathrm{master bias}}{\mathrm{master flat} /  \langle\mathrm{master flat}\rangle }
$$

In [47]:
# Einlesen eines FITS-Bildes mit Header, Header enthält Metadata, Data als Matrix gespeichert
################################
#import numpy as np
#from astropy.io import fits
#data, header = fits.getdata("/path/to/file.fits", header=True)

# Korrektur mittels Biasframe (Subtraktion)
#data = fits.getdata("/path/to/file.fits")
#bias = fits.getdata("/path/to/bias.fits")
#data_minus_bias = data - bias
###

#ist nur fürs einzelne einlesen

In [48]:
# Finden von FITS-Bildern und Einlesen mittels List Comprehension. liest alle fit datein ein
from glob import glob
import numpy as np
from astropy.io import fits
np.set_printoptions(threshold=10000000)#too many elements otherwise ipynb shows ....
#hier haben wir jetzt alle verschiedenen datenfiles in den verschiedenen data lists.
pathsbias_images = sorted(glob(r"C:\Users\raffi\Desktop\Praktikum-main\gruppe6*bias.fit"))
biasdata_images = [fits.getdata(path) for path in pathsbias_images]
pathsdark_images = sorted(glob(r"C:\Users\raffi\Desktop\Praktikum-main\gruppe6*dark.fit"))
darkdata_images = [fits.getdata(path) for path in pathsdark_images]
pathsflat_images = sorted(glob(r"C:\Users\raffi\Desktop\Praktikum-main\gruppe6*flat.fit"))
flatdata_images = [fits.getdata(path) for path in pathsflat_images]
pathsThAr_images = sorted(glob(r"C:\Users\raffi\Desktop\Praktikum-main\gruppe6*ThAr.fit"))
ThArdata_images = [fits.getdata(path) for path in pathsThAr_images]
pathsfilter_images = sorted(glob(r"C:\Users\raffi\Desktop\Praktikum-main\gruppe6-0001_filter*fit"))
filterdata_images = [fits.getdata(path) for path in pathsfilter_images]
print(len(biasdata_images)) #5
print(len(darkdata_images)) #5
print(len(flatdata_images) )#5
print(len(ThArdata_images)) #5
print(len(filterdata_images)) #2

#avg values per pixel von den daten
avg_values_thar= sum(ThArdata_images[:5]) / 5
avg_values_flat= sum(flatdata_images[:5]) / 5
avg_values_dark= sum(darkdata_images[:5]) / 5
avg_values_bias= sum(biasdata_images[:5]) / 5

#test ob sum passt, vom ersten pixel zumindest
#print(ThArdata_images[0][0][0])
#print(ThArdata_images[1][0][0])
#print(ThArdata_images[2][0][0])
#print(ThArdata_images[3][0][0])
#print(ThArdata_images[4][0][0])

#correctur
values_corrected=avg_values_thar-avg_values_bias-avg_values_dark
values_flat_corrected=values_corrected/avg_values_flat

#test ob das corrected value passt
#print(avg_values_thar[0][0])
#print(avg_values_bias[0][0])
#print(avg_values_thar[0][0]-avg_values_bias[0][0])
#print(values_bias_corrected[0])

indices_non_negative_rows = np.where(np.any(values_flat_corrected >= 0, axis=1))[0] #bekommen dadurch die zeilen wo zumindest ein wert nicht negative ist

#print(indices_non_negative_rows) 337 till 395 should be thar lamp  and 478 till 579 should be sun
data_cutout_thar = values_flat_corrected[337:395]
data_cutout_thar[data_cutout_thar < 0] = np.nan #sets all values that are negative in the row to nan so that they wont be summed up later
spectrum_thar = np.nanmedian(data_cutout_thar, axis=0) #sums up all values in a column I think and then calculates the mean 
print(spectrum_thar)

#should be the data of sun 
data_cutout_sun = values_flat_corrected[478:579]
data_cutout_sun[data_cutout_sun < 0] = np.nan
spectrum_sun = np.nanmedian(data_cutout_sun, axis=0)
print(spectrum_sun)


np.set_printoptions(threshold=1000)

5
5
5
5
2
[       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        

C:\Users\raffi\AppData\Local\Temp\ipykernel_17152\3309341830.py:51: RuntimeWarning: All-NaN slice encountered
  spectrum_thar = np.nanmedian(data_cutout_thar, axis=0) #sums up all values in a column I think and then calculates the mean


## 2. Stacken, Extrahieren 1D Spektrum

Erstellung von gemittelten Aufnahmen zur Verbesserung des Signals und zum Zweck der Verwerfung von Ausreißern (z.B. Cosmics).

In [49]:
# Berechnung des Medians für jede Pixelsäule (z.B. für Masterbias)
data_median = np.nanmedian(data_images, axis=0)

NameError: name 'data_images' is not defined

Extraktion eines 1-dimensionalen Spektrums: Von den kalibrierten Aufnahmen kann ein 1-dimensionales Spektrum extrahiert werden, indem ein Rechteck an der passenden Stelle ausgeschnitten wird. Darauf folgend berechnet man den Median entlang einer Pixelsäule.

In [ ]:
# Ausschneiden eines Rechtecks, basierend auf Pixelkoordinaten
# Achten Sie auf die richtige Reihenfolge der x/y-Koordinaten!
data_cutout = data[100:250, 300:320]

# Alternativ entlang der gesamten x-Achse
data_cutout = data[100:250, :]

# Median entlang des Spalts für alle Pixel in Dispersionsrichtung
spectrum_flat = np.nanmedian(data_cutout, axis=0)

# Speichern als FITS-Tabelle
from astropy.table import Table
table = Table(data=[np.arange(len(spectrum_flat)), spectrum_flat], names=["Pixel", "Flux (ADU)"])
table.write("/path/to/table.fits")

## 3. Kombinierte Flat Korrektur

Idee: Das Flat das durch den Spektrographen aufgenommen wird enthält das Black-Body Spektrum der Tischlampe. Um dieses auszukorrigieren, nehmen wir ein zusätzliches Flat auf, das bereits korrigiert ist (Silva-Nova Spektrometer), durch welches das Spektrograph Flat dividiert werden soll. 

Dafür ist zu beachten dass das Wellenlängen-sampling der beiden Flats anders ist. Kontrollieren Sie nach in welchen Wellenlängbereichen die beiden Flats aufgenommen wurden. 

Tip: Mit Interpolieren (zB `interp1d`) kann man das erwünschte sampling für das Nova Spektrometer erhalten.

## 4. Wellenlängekalibration
Identifikation der ThAr-Linien. Zur Identifikation der Linien soll der beigelegte Linienatlas verwendet werden. Beachten Sie, dass das aufgenommene Spektrum Wellenlängen von 4200 bis 5500 abdeckt.

Für das finden der peaks kann zB von `scipy.signal` die funktion `find_peaks`verwendet werden.


Erstellung einer Wellenlängenkalibration auf Basis der identifizierten Linien. Hierfür erstellen Sie einen Fit zwischen Pixelkoordinaten und Wellenlänge der identifizierten Linien. Sie können hierfür numpy verwenden

In [ ]:
# Fit zwischen Pixel und Wellenlänge mittels Polynom 2. Ordnung
coeff = np.polynomial.Polynomial.fit(pixel, wavelength, 2)

NameError: name 'pixel' is not defined

Die so erstellte Interpolation kann nun verwendet werden, um die Wellenlängenkalibration auf das Sonnenspektrum anzuwenden.

## 5. Spektrale Auflösung

Bestimmung der spektralen Auflösung aus der Breite der ThAr-Linien. Die Auflösung eines Spektrographen ist definiert durch
$$
        R \equiv \frac{\lambda}{\Delta \lambda},
$$
wobei hier R die Auflösung des Spektrographen darstellt, $\lambda$ die beobachtete Wellenlänge und $\Delta \lambda$ den kleinsten Unterschied in der Wellenlänge, den der Spektrograph auflösen kann. In der Praxis kann man für $\Delta \lambda$ die Full Width Half Maximum (FWHM) einer Linie bestimmen.

## 6. Signal-to-Noise Ratio

Abschätzung des Signal-Rausch-Verhältnisses (S/N) des Sonnenspektrums. Das S/N bei Spektren ist wegen unterschiedlicher Linienstärke wellenlängenabhängig. Die Angabe eines einzelnen S/N-Wertes für ein gesamtes Spektrum ist daher meistens nicht aussagekräftig. Zur Vereinfachung nehmen Sie an, dass die dominierende Rauschkomponente das Signal selbst ist, d.h. das Rauschen folgt einer Poissonverteilung nur abhängig vom Signal (den Gain der Kamera finden Sie im FITS-Header).